In [1]:
%load_ext autoreload
%autoreload 2
import torch
import numpy as np
import pandas as pd
from omegaconf import OmegaConf
from pathlib import Path
from pprint import pprint
# plotting
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
# mycode
from repo import REPO
from ml_utilities.output_loader.job_output import SweepResult, JobResult
from ml_utilities.output_loader import create_job_output_loader
from ml_utilities.output_loader.plot import plot_sweep_summary, plot_data_log_values

# 11.5.1 MNIST lenet pretrain

**RERUN 11.5.0**: Run with a new seed. 

Do mode connectivity experiments with different seeds.

## A. Experiment config

### Config file

In [2]:
config_yaml = """
run_config:
  exec_type: parallel # sequential
  hostname: raptor
  gpu_ids: [0]
  runs_per_gpu: 2

  wandb: # wandb config for run_handler, if "wandb: null" then logging to wandb is disabled for run_handler
    init:
      tags:
        - ${config.experiment_data.experiment_tag}_exps
        - run_handler
      notes: #
      group: ${config.experiment_data.experiment_tag}
      job_type: run_handler

seeds: [1,2]

sweep:
  type: grid
  axes:
    - parameter: data.dataset_kwargs.rotation_angle
      vals: [0.0] #linspace(0,180,360,endpoint=True)

start_num: 1 # use this to count how often this config is run

###
config:
  experiment_data:
    entity: jkuiml-fsl
    project_name: sparsity
    experiment_tag: "11.5"
    experiment_type: startnum_${start_num}
    experiment_name: mnist-${config.experiment_data.experiment_tag}.${start_num}-lenet
    experiment_dir: null
    experiment_notes: Different random inits for mode connectivity analysis.
    job_name: null
    seed: 0
    hostname: null # the server on which the run is run, will be filled by run_handler
    gpu_id: 5

  wandb:
    init:
      tags: # list(), used to tag wandblogger
        - ${config.experiment_data.experiment_tag}_exps
      notes: ${config.experiment_data.experiment_notes} # str, used to make notes to wandblogger
      group: ${config.experiment_data.experiment_tag} # null
      job_type: ${config.experiment_data.experiment_type} # examples: hypsearch, pretrain, eval, etc.

    watch:
      log: null #parameters #null #all
      log_freq: 5000

  model:
    name: fc
    model_kwargs:
      input_size: 784
      hidden_sizes:
        - 300
        - 100
      output_size: 10
      flatten_input: True
      dropout: null
      act_fn: relu

  trainer:
    training_setup: supervised
    n_steps: 2000
    log_train_step_every: 1
    log_additional_train_step_every_multiplier: 1
    log_additional_logs: True
    val_every: 5
    save_every: 5
    early_stopping_patience: 200
    batch_size: 128
    optimizer_scheduler:
      optimizer_name: adamw #sgd #adamw
      optimizer_kwargs:
        lr: 0.001
        weight_decay: 0.0
    init_model: null

    loss: crossentropy

    metrics:
      - Accuracy
    num_workers: 4
    verbose: False

  data:
    dataset: rotatedvision
    dataset_kwargs:
      data_root_path: /system/user/beck/pwbeck/data
      dataset: mnist
      rotation_angle: 0.0
    dataset_split:
      train_val_split: 0.8

"""
cfg = OmegaConf.create(config_yaml)

In [3]:
run_command = REPO.create_experiment(cfg, override=False)
print(run_command)

python run_sweep.py --config-name mnist-11.5.1-lenet.yaml


### Sweep result

In [ ]:
sweepr = REPO.get_output_loader(cfg)
print(sweepr)

In [ ]:
# failed_jobs = sweepr.get_failed_jobs()
# failed_jobs # No failed jobs

## B. Analysis 1

In [ ]:
pprint(sweepr.available_log_columns)

In [ ]:
sw_summary = sweepr.get_summary()

In [ ]:
sw_summary